In [3]:
import os
from PIL import Image
import re
from tqdm import tqdm 
from spellchecker import SpellChecker
import re
import pytesseract
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForCausalLM
import os
import torch
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = "mps" if torch.backends.mps.is_available() else "cpu"

Grammarly

In [ ]:
# Set TOKENIZERS_PARALLELISM to true

tokenizer = AutoTokenizer.from_pretrained("grammarly/coedit-xl")
model = T5ForConditionalGeneration.from_pretrained("grammarly/coedit-xl")

def correct_grammar(text):
# Encode the text input
text = "Fix grammatical errors in this sentence: " + text
input_ids = tokenizer(text, return_tensors="pt").input_ids
# Generate corrected text output
outputs = model.generate(input_ids, max_length=8192)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "mps" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 1bbb6689-12d9-4094-8fd2-5b8366c54c02)')' thrown while requesting HEAD https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Check if MPS (Apple's Metal Performance Shaders) is available and set the device accordingly
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Load the tokenizer and model
print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained("teknium/OpenHermes-2.5-Mistral-7B")
model = AutoModelForCausalLM.from_pretrained("teknium/OpenHermes-2.5-Mistral-7B").to(device)
print("Tokenizer and model loaded successfully.")

def correct_grammar(text):
    print(f"Correcting grammar for text: {text}")

    # Format the prompt for grammar correction
    prompt = f"Correct the grammar: {text}"
    print(f"Formatted prompt: {prompt}")

    # Encode the text input into model's input format
    print("Encoding text...")
    model_inputs = tokenizer(prompt, return_tensors="pt").to(device)
    print("Text encoded successfully.")

    # Generate corrected text output
    print("Generating output...")
    with torch.no_grad():
        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
    print("Output generated.")

    # Decode the generated ids to text
    print("Decoding output...")
    corrected_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f"Decoded text: {corrected_text}")

    # Remove the initial prompt part from the output and return the corrected text
    corrected_text = corrected_text.replace(prompt, "").strip()
    print(f"Final corrected text: {corrected_text}")

    return corrected_text

# Example usage
corrected_text = correct_grammar("I has a book.")
print(f"Corrected Text: {corrected_text}")


In [ ]:

# Decode and return the corrected text
corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
return corrected_text
spell = SpellChecker()

spell = SpellChecker()



def assess_ocr_quality(text):
# Number Heavy Check
num_ratio = sum(c.isdigit() for c in text) / len(text) if text else 0
if num_ratio > 0.5:
    return 'number_heavy'

# Basic Length Check
if len(text) < 20:  # arbitrary minimum length
    return 'unusable'

# Spell Checking and Basic Grammar Analysis
words = text.split()
misspelled = spell.unknown(words)
if len(misspelled) > 0.4 * len(words):  # if more than 40% of words are misspelled
    return 'unusable'

# Simple Coherence Check (can be improved with NLP techniques)
# Example: Check for frequent occurrences of nonsensical character combinations
if re.search(r'[^\w\s]', text):  # regex for non-word, non-space characters
    return 'okay'

# If none of the above, classify as good
return 'good'

source_dir = "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/images"
target_dirs = {
'good': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/good_ocr",
'okay': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/okay_ocr",
'unusable': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/unusable_ocr",
'number_heavy': "/Users/garfieldgreglim/Documents/JQ/Knowledgebase 2/image_ocrs/number_heavy_ocrs"
}
if not os.path.exists(source_dir):
print("Source directory does not exist.")
exit(1)

for dir in target_dirs.values():
if not os.path.exists(dir):
    os.makedirs(dir)

# Get list of image files
image_files = [f for f in os.listdir(source_dir) if f.lower().endswith((".jpg", ".png"))]

# Process each file with a progress bar
for filename in tqdm(image_files, desc="Processing images"):
image_path = os.path.join(source_dir, filename)
try:
    text = pytesseract.image_to_string(Image.open(image_path))
    text = correct_grammar(text)
    quality = assess_ocr_quality(text)
    with open(os.path.join(target_dirs[quality], filename + ".txt"), "w") as text_file:
        text_file.write(text)
except Exception as e:
    print(f"Error processing {filename}: {e}")